<a href="https://colab.research.google.com/github/luck058/yfinance-prediction/blob/main/import_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install yfinance

In [2]:
import yfinance as yf
import pandas as pd

In [3]:
def help():
    print('# Set up data fetcher using:')
    print('data_fetcher = YahooFinanceDataFetcher(self, ticker_symbol, period="max", interval="1d")\n')
    print('# Get historical data using:')
    print('df_data = data_fetcher.get_data(verbose=False)\n')
    print('# Get financials using:')
    print('df_financials = data_fetcher.get_financials(verbose=False)\n')
    print('# Get actions using:')
    print('df_actions = data_fetcher.get_actions(verbose=False)\n')

In [24]:
class YahooFinanceDataFetcher:
    """All functions defined in this notebook are added to this class"""
    def __init__(self, ticker_symbol, period="max", interval="1d", train_frac=0.8):
        self.ticker_symbol = ticker_symbol
        self.ticker = yf.Ticker(ticker_symbol)
        self.period = period
        self.interval = interval
        # Fraction of the data which will be used for training (last section will be reserved for testing)
        self.train_frac = train_frac
        self.X = None
        self.y = None

In [28]:
def get_raw_data(self, verbose=False):
    """Get historical data from Yahoo Finance. If reload=False, gets saved data from GitHub if available"""
    data = self.ticker.history(period=self.period, interval=self.interval)
    df_data = pd.DataFrame(data)

    if verbose:
        display("Historical Data:")
        display(df_data.head())

    return df_data

YahooFinanceDataFetcher.get_raw_data = get_raw_data

In [25]:
def get_financials(self, verbose=False):
    financials = self.ticker.financials
    df_financials = pd.DataFrame(financials).T

    if verbose:
        print("Financials:")
        display(df_financials)

    return df_financials

YahooFinanceDataFetcher.get_financials = get_financials

In [26]:
def get_actions(self, verbose=False):
    actions = self.ticker.actions
    df_actions = pd.DataFrame(actions)

    if verbose:
        print("Actions:")
        display(df_actions)

    return df_actions

YahooFinanceDataFetcher.get_actions = get_actions

In [31]:
def get_X_y(self, reload=False, test=False, verbose=False
            ):
    """Return X and y dataframes"""
    # If the X and y have already been created
    if self.X is not None and self.y is not None:
        train_length = int(len(self.X) * self.train_frac)
        test_length = len(self.X) - train_length
        if test:
            return self.X.tail(test_length), self.y.tail(test_length)
        else:
            return self.X.head(train_length), self.y.head(train_length)

    df = self.get_raw_data(verbose=verbose)
    df
    # display(df)

    return None, None





YahooFinanceDataFetcher.get_X_y = get_X_y

data_fetcher = YahooFinanceDataFetcher("AAPL")

X, y = data_fetcher.get_X_y(verbose=True)

display(X, y)

'Historical Data:'

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12 00:00:00-05:00,0.098834,0.099264,0.098834,0.098834,469033600,0.0,0.0
1980-12-15 00:00:00-05:00,0.094108,0.094108,0.093678,0.093678,175884800,0.0,0.0
1980-12-16 00:00:00-05:00,0.087232,0.087232,0.086802,0.086802,105728000,0.0,0.0
1980-12-17 00:00:00-05:00,0.088951,0.089381,0.088951,0.088951,86441600,0.0,0.0
1980-12-18 00:00:00-05:00,0.091530,0.091959,0.091530,0.091530,73449600,0.0,0.0


None

None

In [ ]:
# data_fetcher = YahooFinanceDataFetcher("AAPL", period="1d")

# df_data = data_fetcher.get_data()

# display(df_data)

An error occurred in get_data: [Errno 2] No such file or directory: 'AAPL-1d-1d.csv'


# Archive

In [19]:
# def save_data(self, verbose=False):
#     """Saves data to csv file. Saves as 'ticker-period-interval.csv'"""
#     file_name = f"{self.ticker_symbol}-{self.period}-{self.interval}.csv"
#     df = self.get_raw_data(reload=True, verbose=False)
#     df.to_csv(file_name)
#     if verbose:
#         print(f"Saved data as {file_name}")

# YahooFinanceDataFetcher.save_data = save_data

In [13]:
# def get_raw_data(self, reload=False, verbose=False):
#     """Get historical data from Yahoo Finance. If reload=False, gets saved data from GitHub if available"""
#     if not reload:
#         # Try in order: get data from current file structure, get data from GitHub, get data from yfinance
#         try:
#             if verbose:
#                 print("Loading saved data")
#             data = pd.read_csv(f"{self.ticker_symbol}-{self.period}-{self.interval}.csv")
#         except FileNotFoundError:
#             try:
#                 if verbose:
#                     print("No saved data found. Connecting to GitHub")
#                 !git clone https://github.com/luck058/yfinance-prediction
#                 %cd yfinance-prediction
#                 data = pd.read_csv(f"{self.ticker_symbol}-{self.period}-{self.interval}.csv")
#             except FileNotFoundError:
#                 if verbose:
#                     print("No saved data found. Reloading data from Yahoo Finance")
#                 data = self.ticker.history(period=self.period, interval=self.interval)
#     else:
#         if verbose:
#             print("Reloading data from Yahoo Finance. This will not be saved. Use save_data() to save to GitHub")
#         data = self.ticker.history(period=self.period, interval=self.interval)

#     df_data = pd.DataFrame(data)

#     if verbose:
#         display("Historical Data:")
#         display(df_data.head())

#     return df_data

# YahooFinanceDataFetcher.get_raw_data = get_raw_data